In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
from PIL import Image
from behavioral_analysis.pandas_tools.files import build_file_df
from behavioral_analysis.visualization.cv2_funcs import perspective_transform_arena, add_rgba_overlay_to_frame, fig2cv2
from behavioral_analysis.visualization.matplotlib_funcs import create_canvas
from behavioral_analysis.visualization.plots import plot_arena, load_arena_from_json
from behavioral_analysis.visualization.video_funcs import get_frames, get_cv2_video_properties
from tqdm.cli import tqdm

In [11]:
root_dir = pathlib.Path(r"D:\Folder With Backup\2019_paper\data")
plot_dir = root_dir / "plot_output"
arena_frames_dir = plot_dir / "verification" / "arena_frames"
annotations_dir = root_dir / "annotations"
videos_dir = root_dir / "videos_remuxed"

# Looming Stimulus

## Old Data

In [54]:
local_annotations_dir = annotations_dir / "looming_stimulus" / "cohort1"
expanded_arena_transforms = pd.read_csv(local_annotations_dir / "expanded_arena_transforms.csv", index_col=[0, 1, 2])
arena_dict = load_arena_from_json(local_annotations_dir / "arena_dict.json")
expanded_arena_transforms

0         1      2         3  \
mouse_id loom_id selection_frame_index                                        
1_1      0       0                      1.176142 -1.155054  331.0 -0.110053   
         1       0                      1.176142 -1.155054  331.0 -0.110053   
         2       0                      1.176142 -1.155054  331.0 -0.110053   
         3       0                      1.176142 -1.155054  331.0 -0.110053   
         4       0                      1.176142 -1.155054  331.0 -0.110053   
...                                          ...       ...    ...       ...   
3_4      10      0                      1.187783 -1.149983  325.0 -0.115431   
         11      0                      1.187783 -1.149983  325.0 -0.115431   
         12      0                      1.187783 -1.149983  325.0 -0.115431   
         13      0                      1.187783 -1.149983  325.0 -0.115431   
         14      0                      1.187783 -1.149983  325.0 -0.115431   

                                               4      5         6         7  \
mouse_id loom_id selection_frame_index                                        
1_1      0       0                     -1.013749  755.0 -0.000103 -0.001739   
         1       0                     -1.013749  755.0 -0.000103 -0.001739   
         2       0                     -1.013749  755.0 -0.000103 -0.001739   
         3       0                     -1.013749  755.0 -0.000103 -0.001739   
         4       0                     -1.013749  755.0 -0.000103 -0.001739   
...                                          ...    ...       ...       ...   
3_4      10      0                     -1.007834  759.0 -0.000100 -0.001725   
         11      0                     -1.007834  759.0 -0.000100 -0.001725   
         12      0                     -1.007834  759.0 -0.000100 -0.001725   
         13      0                     -1.007834  759.0 -0.000100 -0.001725   
         14      0                     -1.007834  759.0 -0.000100 -0.001725   

                                          8  
mouse_id loom_id selection_frame_index       
1_1      0       0                      1.0  
         1       0                      1.0  
         2       0                      1.0  
         3       0                      1.0  
         4       0                      1.0  
...                                     ...  
3_4      10      0                      1.0  
         11      0                      1.0  
         12      0                      1.0  
         13      0                      1.0  
         14      0                      1.0  

[171 rows x 9 columns]

In [55]:
local_videos_dir = videos_dir / "looming_stimulus" / "cohort1"
video_file_df = build_file_df(str(local_videos_dir), r"(?P<date_string>\d+)_mouse(?P<mouse_id>[\d_]+)_loom(?P<loom_id>\d+)_remuxed.mp4")
video_file_df = video_file_df.astype({"loom_id": int})
video_file_df = video_file_df.set_index(["mouse_id", "loom_id"])
video_file_df

date_string  \
mouse_id loom_id               
1_1      0          20210513   
         10         20210513   
         11         20210513   
         12         20210513   
         13         20210513   
...                      ...   
3_4      5          20210516   
         6          20210516   
         7          20210516   
         8          20210516   
         9          20210516   

                                                          file_path  
mouse_id loom_id                                                     
1_1      0        D:\Folder With Backup\2019_paper\data\videos_r...  
         10       D:\Folder With Backup\2019_paper\data\videos_r...  
         11       D:\Folder With Backup\2019_paper\data\videos_r...  
         12       D:\Folder With Backup\2019_paper\data\videos_r...  
         13       D:\Folder With Backup\2019_paper\data\videos_r...  
...                                                             ...  
3_4      5        D:\Folder With Backup\2019_paper\data\videos_r...  
         6        D:\Folder With Backup\2019_paper\data\videos_r...  
         7        D:\Folder With Backup\2019_paper\data\videos_r...  
         8        D:\Folder With Backup\2019_paper\data\videos_r...  
         9        D:\Folder With Backup\2019_paper\data\videos_r...  

[161 rows x 2 columns]

In [58]:
local_arena_frames_dir = arena_frames_dir / "looming_stimulus" / "cohort1"
local_arena_frames_dir.mkdir(parents=True, exist_ok=True)

for (mouse_id, loom_id, selection_frame_index), flattened_arena_transform in tqdm(expanded_arena_transforms.iterrows(), total=len(expanded_arena_transforms)):
    if not (mouse_id, loom_id) in video_file_df.index:
        continue
    
    video_path = video_file_df.loc[(mouse_id, loom_id), "file_path"]
    total_frame_count = int(get_cv2_video_properties(video_path, cv2.CAP_PROP_FRAME_COUNT)[0])
    selection_frame_indices = [selection_frame_index, int(total_frame_count*0.9)]
    for selection_frame_index in selection_frame_indices:
        verification_frame = get_frames(video_path, [selection_frame_index], jump_to_frames=True)[0]

        arena_transform = np.reshape(flattened_arena_transform, (3, 3))
        transformed_arena = perspective_transform_arena(arena_dict, arena_transform)
        canvas = create_canvas(*verification_frame.shape[:2][::-1], dpi=100)
        plot_arena(transformed_arena, ax=canvas.gca())

        annotated_verification_frame = add_rgba_overlay_to_frame(verification_frame.copy(), fig2cv2(canvas))
        plt.close(canvas)

        Image.fromarray(annotated_verification_frame).save(local_arena_frames_dir / pathlib.Path(video_path).name.replace(".mp4", f"_frame{selection_frame_index}_verification.png"))

100%|██████████| 171/171 [01:32<00:00,  1.85it/s]


## New Data

In [59]:
local_annotations_dir = annotations_dir / "looming_stimulus" / "cohort2"
expanded_arena_transforms = pd.read_csv(local_annotations_dir / "expanded_arena_transforms.csv", index_col=[0, 1, 2])
arena_dict = load_arena_from_json(local_annotations_dir / "arena_dict.json")
expanded_arena_transforms

0         1      2         3  \
mouse_id loom_id selection_frame_index                                        
1_1      0       0                      1.617297 -0.772364  241.0  0.016154   
         1       0                      1.617297 -0.772364  241.0  0.016154   
         2       0                      1.617297 -0.772364  241.0  0.016154   
         3       0                      1.617297 -0.772364  241.0  0.016154   
         4       0                      1.617297 -0.772364  241.0  0.016154   
...                                          ...       ...    ...       ...   
3_9      9       0                      1.634595 -0.763804  242.0  0.034918   
         10      0                      1.634595 -0.763804  242.0  0.034918   
         11      0                      1.634595 -0.763804  242.0  0.034918   
         12      0                      1.634595 -0.763804  242.0  0.034918   
         13      0                      1.634595 -0.763804  242.0  0.034918   

                                               4      5         6         7  \
mouse_id loom_id selection_frame_index                                        
1_1      0       0                     -0.647631  723.0  0.000008 -0.001278   
         1       0                     -0.647631  723.0  0.000008 -0.001278   
         2       0                     -0.647631  723.0  0.000008 -0.001278   
         3       0                     -0.647631  723.0  0.000008 -0.001278   
         4       0                     -0.647631  723.0  0.000008 -0.001278   
...                                          ...    ...       ...       ...   
3_9      9       0                     -0.673110  755.0  0.000035 -0.001269   
         10      0                     -0.673110  755.0  0.000035 -0.001269   
         11      0                     -0.673110  755.0  0.000035 -0.001269   
         12      0                     -0.673110  755.0  0.000035 -0.001269   
         13      0                     -0.673110  755.0  0.000035 -0.001269   

                                          8  
mouse_id loom_id selection_frame_index       
1_1      0       0                      1.0  
         1       0                      1.0  
         2       0                      1.0  
         3       0                      1.0  
         4       0                      1.0  
...                                     ...  
3_9      9       0                      1.0  
         10      0                      1.0  
         11      0                      1.0  
         12      0                      1.0  
         13      0                      1.0  

[340 rows x 9 columns]

In [60]:
local_videos_dir = videos_dir / "looming_stimulus" / "cohort2"
video_file_df = build_file_df(str(local_videos_dir), r"(?P<date_string>\d+)_mouse(?P<mouse_id>[\d_]+)_loom(?P<loom_id>\d+)_remuxed.mp4")
video_file_df = video_file_df.astype({"loom_id": int})
video_file_df = video_file_df.set_index(["mouse_id", "loom_id"])
video_file_df

date_string  \
mouse_id loom_id               
1_1      0          20241030   
         10         20241030   
         11         20241030   
         12         20241030   
         13         20241030   
...                      ...   
3_12     5          20241109   
         6          20241109   
         7          20241109   
         8          20241109   
         9          20241109   

                                                          file_path  
mouse_id loom_id                                                     
1_1      0        D:\Folder With Backup\2019_paper\data\videos_r...  
         10       D:\Folder With Backup\2019_paper\data\videos_r...  
         11       D:\Folder With Backup\2019_paper\data\videos_r...  
         12       D:\Folder With Backup\2019_paper\data\videos_r...  
         13       D:\Folder With Backup\2019_paper\data\videos_r...  
...                                                             ...  
3_12     5        D:\Folder With Backup\2019_paper\data\videos_r...  
         6        D:\Folder With Backup\2019_paper\data\videos_r...  
         7        D:\Folder With Backup\2019_paper\data\videos_r...  
         8        D:\Folder With Backup\2019_paper\data\videos_r...  
         9        D:\Folder With Backup\2019_paper\data\videos_r...  

[336 rows x 2 columns]

In [61]:
local_arena_frames_dir = arena_frames_dir / "looming_stimulus" / "cohort2"
local_arena_frames_dir.mkdir(parents=True, exist_ok=True)

for (mouse_id, loom_id, selection_frame_index), flattened_arena_transform in tqdm(expanded_arena_transforms.iterrows(), total=len(expanded_arena_transforms)):
    if not (mouse_id, loom_id) in video_file_df.index:
        continue
    
    video_path = video_file_df.loc[(mouse_id, loom_id), "file_path"]
    total_frame_count = int(get_cv2_video_properties(video_path, cv2.CAP_PROP_FRAME_COUNT)[0])
    selection_frame_indices = [selection_frame_index, int(total_frame_count*0.9)]
    for selection_frame_index in selection_frame_indices:
        verification_frame = get_frames(video_path, [selection_frame_index], jump_to_frames=True)[0]

        arena_transform = np.reshape(flattened_arena_transform, (3, 3))
        transformed_arena = perspective_transform_arena(arena_dict, arena_transform)
        canvas = create_canvas(*verification_frame.shape[:2][::-1], dpi=100)
        plot_arena(transformed_arena, ax=canvas.gca())

        annotated_verification_frame = add_rgba_overlay_to_frame(verification_frame.copy(), fig2cv2(canvas))
        plt.close(canvas)

        Image.fromarray(annotated_verification_frame).save(local_arena_frames_dir / pathlib.Path(video_path).name.replace(".mp4", f"_frame{selection_frame_index}_verification.png"))

100%|██████████| 340/340 [03:35<00:00,  1.58it/s]


# Rat Odor

## Old Data

In [64]:
local_annotations_dir = annotations_dir / "rat_odor" / "cohort1"
expanded_arena_transforms = pd.read_csv(local_annotations_dir / "expanded_arena_transforms.csv", index_col=[0, 1, 2])
arena_dict = load_arena_from_json(local_annotations_dir / "arena_dict.json")
expanded_arena_transforms

0         1      2  \
mouse_id phase       selection_frame_index                              
3_4      WithOdor    0                      1.418397 -0.092215  296.0   
         WithoutOdor 0                      1.409932 -0.084658  294.0   
3_3      WithOdor    0                      1.383895 -0.067953  294.0   
         WithoutOdor 0                      1.394449 -0.058399  287.0   
3_2      WithOdor    0                      1.385803 -0.093808  293.0   
         WithoutOdor 0                      1.344420 -0.037864  285.0   
3_1      WithOdor    0                      1.373840 -0.081056  292.0   
         WithoutOdor 0                      1.377633 -0.031653  296.0   
2_4      WithOdor    0                      1.432629 -0.072519  292.0   
         WithoutOdor 0                      1.367063 -0.058698  286.0   
2_3      WithOdor    0                      1.395127 -0.072329  290.0   
         WithoutOdor 0                      1.406303 -0.064639  288.0   
2_2      WithOdor    0                      1.387409 -0.052051  287.0   
         WithoutOdor 0                      1.391807 -0.025996  294.0   
2_1      WithOdor    0                      1.368783 -0.071187  291.0   
         WithoutOdor 0                      1.338011 -0.005916  292.0   
1_4      WithOdor    0                      1.387034 -0.076744  293.0   
         WithoutOdor 0                      1.372832 -0.074406  287.0   
1_3      WithOdor    0                      1.403140 -0.067181  294.0   
         WithoutOdor 0                      1.406046 -0.033593  286.0   
1_2      WithOdor    0                      1.370911 -0.074538  288.0   
         WithoutOdor 0                      1.340342 -0.031289  294.0   
1_1      WithOdor    0                      1.324722 -0.065867  288.0   
         WithoutOdor 0                      1.367678 -0.031152  290.0   

                                                   3         4      5  \
mouse_id phase       selection_frame_index                              
3_4      WithOdor    0                      0.045575  1.364679  319.0   
         WithoutOdor 0                      0.043881  1.341973  303.0   
3_3      WithOdor    0                      0.017868  1.357480  325.0   
         WithoutOdor 0                     -0.015699  1.344992  309.0   
3_2      WithOdor    0                      0.038764  1.358590  302.0   
         WithoutOdor 0                     -0.039951  1.346442  299.0   
3_1      WithOdor    0                      0.023689  1.371934  302.0   
         WithoutOdor 0                     -0.049177  1.313283  290.0   
2_4      WithOdor    0                      0.043090  1.372122  325.0   
         WithoutOdor 0                     -0.024793  1.328794  307.0   
2_3      WithOdor    0                      0.027635  1.357865  321.0   
         WithoutOdor 0                      0.006093  1.362572  301.0   
2_2      WithOdor    0                      0.001178  1.362298  302.0   
         WithoutOdor 0                     -0.044429  1.329333  290.0   
2_1      WithOdor    0                      0.006543  1.352281  297.0   
         WithoutOdor 0                     -0.094521  1.311371  298.0   
1_4      WithOdor    0                      0.020245  1.354308  323.0   
         WithoutOdor 0                     -0.005150  1.332139  303.0   
1_3      WithOdor    0                      0.024379  1.364572  319.0   
         WithoutOdor 0                     -0.021639  1.355386  301.0   
1_2      WithOdor    0                     -0.002962  1.329458  295.0   
         WithoutOdor 0                     -0.065208  1.301156  292.0   
1_1      WithOdor    0                     -0.030324  1.315870  297.0   
         WithoutOdor 0                     -0.057014  1.309991  295.0   

                                                       6         7    8  
mouse_id phase       selection_frame_index                               
3_4      WithOdor    0                      1.644595e-05 -0.000072  1.0  
         WithoutOdor 0         

In [65]:
local_videos_dir = videos_dir / "rat_odor" / "cohort1"
video_file_df = build_file_df(str(local_videos_dir), r"(?P<date_string>\d+)_mouse(?P<mouse_id>[\d_]+)_(?P<phase>\w+)_remuxed.mp4")
video_file_df = video_file_df.set_index(["mouse_id", "phase"])
video_file_df

date_string  \
mouse_id phase                     
1_1      WithOdor       20210528   
         WithoutOdor    20210528   
1_2      WithOdor       20210528   
         WithoutOdor    20210528   
2_1      WithOdor       20210528   
         WithoutOdor    20210528   
2_2      WithOdor       20210528   
         WithoutOdor    20210528   
3_1      WithOdor       20210528   
         WithoutOdor    20210528   
3_2      WithOdor       20210528   
         WithoutOdor    20210528   
1_3      WithOdor       20210529   
         WithoutOdor    20210529   
1_4      WithOdor       20210529   
         WithoutOdor    20210529   
2_3      WithOdor       20210529   
         WithoutOdor    20210529   
2_4      WithOdor       20210529   
         WithoutOdor    20210529   
3_3      WithOdor       20210529   
         WithoutOdor    20210529   
3_4      WithOdor       20210529   
         WithoutOdor    20210529   

                                                              file_path  
mouse_id phase                                                           
1_1      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_2      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
2_1      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
2_2      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
3_1      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
3_2      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_3      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_4      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
2_3      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
2_4      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
3_3      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
3_4      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...

In [66]:
local_arena_frames_dir = arena_frames_dir / "rat_odor" / "cohort1"
local_arena_frames_dir.mkdir(parents=True, exist_ok=True)

for (mouse_id, phase, selection_frame_index), flattened_arena_transform in tqdm(expanded_arena_transforms.iterrows(), total=len(expanded_arena_transforms)):
    if not (mouse_id, phase) in video_file_df.index:
        continue
    
    video_path = video_file_df.loc[(mouse_id, phase), "file_path"]
    total_frame_count = int(get_cv2_video_properties(video_path, cv2.CAP_PROP_FRAME_COUNT)[0])
    selection_frame_indices = [selection_frame_index, int(total_frame_count*0.9)]
    for selection_frame_index in selection_frame_indices:
        verification_frame = get_frames(video_path, [selection_frame_index], jump_to_frames=True)[0]

        arena_transform = np.reshape(flattened_arena_transform, (3, 3))
        transformed_arena = perspective_transform_arena(arena_dict, arena_transform)
        canvas = create_canvas(*verification_frame.shape[:2][::-1], dpi=100)
        plot_arena(transformed_arena, ax=canvas.gca())

        annotated_verification_frame = add_rgba_overlay_to_frame(verification_frame.copy(), fig2cv2(canvas))
        plt.close(canvas)

        Image.fromarray(annotated_verification_frame).save(local_arena_frames_dir / pathlib.Path(video_path).name.replace(".mp4", f"_frame{selection_frame_index}_verification.png"))

100%|██████████| 24/24 [00:15<00:00,  1.55it/s]


## New Data

In [86]:
local_annotations_dir = annotations_dir / "rat_odor" / "cohort2"
expanded_arena_transforms = pd.read_csv(local_annotations_dir / "expanded_arena_transforms.csv", index_col=[0, 1, 2])
arena_dict = load_arena_from_json(local_annotations_dir / "arena_dict.json")
expanded_arena_transforms

0         1     2  \
mouse_id phase       selection_frame_index                             
1_1      WithoutOdor 0                      2.306482  0.013990  68.0   
         WithOdor    0                      2.301294  0.003894  81.0   
1_10     WithoutOdor 0                      2.303827  0.025057  77.0   
         WithOdor    0                      2.320151  0.020580  77.0   
1_11     WithoutOdor 0                      2.317919  0.017176  76.0   
         WithOdor    0                      2.275448  0.005464  78.0   
1_12     WithoutOdor 0                      2.312002  0.009898  77.0   
         WithOdor    0                      2.326729  0.038992  83.0   
1_2      WithoutOdor 0                      2.321019  0.025567  65.0   
         WithOdor    0                      2.309648  0.023702  68.0   
1_3      WithoutOdor 0                      2.319534  0.004656  76.0   
         WithOdor    0                      2.314649 -0.000223  66.0   
1_4      WithoutOdor 0                      2.321059  0.000000  64.0   
         WithOdor    0                      2.311183  0.026313  66.0   
1_5      WithoutOdor 0                      2.307471  0.018810  66.0   
         WithOdor    0                      2.303483  0.000710  71.0   
1_6      WithoutOdor 0                      2.344271  0.016389  67.0   
         WithOdor    0                      2.341329  0.004594  71.0   
1_7      WithoutOdor 0                      2.309686  0.020997  67.0   
         WithOdor    0                      2.319497  0.008431  69.0   
1_8      WithoutOdor 0                      2.325780  0.009981  72.0   
         WithOdor    0                      2.313734  0.014590  76.0   
1_9      WithoutOdor 0                      2.322034  0.013221  75.0   
         WithOdor    0                      2.334365  0.013289  77.0   
3_1      WithoutOdor 0                      2.323328  0.000000  32.0   
         WithOdor    0                      2.295499 -0.015551  44.0   
3_10     WithoutOdor 0                      2.181020 -0.002479  84.0   
         WithOdor    0                      2.201283  0.009306  83.0   
3_11     WithoutOdor 0                      2.209295  0.024183  82.0   
         WithOdor    0                      2.214328  0.023970  67.0   
3_12     WithoutOdor 0                      2.207073  0.012054  83.0   
         WithOdor    0                      2.217255  0.015689  83.0   
3_2      WithoutOdor 0                      2.308296 -0.003620  54.0   
         WithOdor    0                      2.294528 -0.014876  63.0   
3_3      WithoutOdor 0                      2.178854  0.005156  73.0   
         WithOdor    0                      2.190856  0.009329  73.0   
3_4      WithoutOdor 0                      2.227300  0.016835  71.0   
         WithOdor    0                      2.203542  0.017786  75.0   
3_5      WithoutOdor 0                      2.190668  0.026789  76.0   
         WithOdor    0                      2.164116  0.027734  70.0   
3_6      WithoutOdor 0                      2.219585  0.028607  78.0   
         WithOdor    0                      2.180811  0.008104  81.0   
3_7      WithoutOdor 0                      2.197279  0.020148  79.0   
         WithOdor    0                      2.213275  0.008085  79.0   
3_8      WithoutOdor 0                      2.160808  0.011738  81.0   
         WithOdor    0                      2.188978  0.011452  82.0   
3_9      WithoutOdor 0                      2.185133  0.015956  81.0   
         WithOdor    0                      2.223796 -0.003384  70.0   

                                                   3         4      5  \
mouse_id phase       selection_frame_index                              
1_1      WithoutOdor 0                     -0.000528  2.240335  246.0   
         WithOdor    0                      0.003575  2.241925  229.0   
1_10     WithoutOdor 0                      0.008748  2.271548  193.0   
         WithOdor    0                      0.016328  2.256190  193.0   
1_11     Wit

In [69]:
_index_df = expanded_arena_transforms.index.to_frame()
_index_df["mouse_id"] = _index_df["mouse_id"].astype(str).str.replace(".", "_")
expanded_arena_transforms.index = pd.MultiIndex.from_frame(_index_df)
expanded_arena_transforms.to_csv(local_annotations_dir / "expanded_arena_transforms.csv")

In [87]:
local_videos_dir = videos_dir / "rat_odor" / "cohort2"
video_file_df = build_file_df(str(local_videos_dir), r"(?P<date_string>\d+)_mouse(?P<mouse_id>[\d_]+)_(?P<phase>\w+)_remuxed.mp4")
video_file_df = video_file_df.set_index(["mouse_id", "phase"])
video_file_df

date_string  \
mouse_id phase                     
1_1      WithOdor       20241113   
         WithoutOdor    20241113   
1_2      WithOdor       20241113   
         WithoutOdor    20241113   
1_3      WithOdor       20241113   
         WithoutOdor    20241113   
1_4      WithOdor       20241113   
         WithoutOdor    20241113   
1_5      WithOdor       20241113   
         WithoutOdor    20241113   
1_6      WithOdor       20241113   
         WithoutOdor    20241113   
1_10     WithOdor       20241114   
         WithoutOdor    20241114   
1_11     WithOdor       20241114   
         WithoutOdor    20241114   
1_12     WithOdor       20241114   
         WithoutOdor    20241114   
1_7      WithOdor       20241114   
         WithoutOdor    20241114   
1_8      WithOdor       20241114   
         WithoutOdor    20241114   
1_9      WithOdor       20241114   
         WithoutOdor    20241114   
3_1      WithOdor       20241120   
         WithoutOdor    20241120   
3_2      WithOdor       20241120   
         WithoutOdor    20241120   
3_3      WithOdor       20241120   
         WithoutOdor    20241120   
3_4      WithOdor       20241120   
         WithoutOdor    20241120   
3_5      WithOdor       20241120   
         WithoutOdor    20241120   
3_6      WithOdor       20241120   
         WithoutOdor    20241120   
3_10     WithOdor       20241121   
         WithoutOdor    20241121   
3_11     WithOdor       20241121   
         WithoutOdor    20241121   
3_12     WithOdor       20241121   
         WithoutOdor    20241121   
3_7      WithOdor       20241121   
         WithoutOdor    20241121   
3_8      WithOdor       20241121   
         WithoutOdor    20241121   
3_9      WithOdor       20241121   
         WithoutOdor    20241121   

                                                              file_path  
mouse_id phase                                                           
1_1      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_2      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_3      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_4      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_5      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_6      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_10     WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_11     WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_12     WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_7      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_8      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
1_9      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
3_1      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  
3_2      WithOdor     D:\Folder With Backup\2019_paper\data\videos_r...  
         WithoutOdor  D:\Folder With Backup\2019_paper\data\videos_r...  


In [88]:
local_arena_frames_dir = arena_frames_dir / "rat_odor" / "cohort2"
local_arena_frames_dir.mkdir(parents=True, exist_ok=True)

for (mouse_id, phase, selection_frame_index), flattened_arena_transform in tqdm(expanded_arena_transforms.iterrows(), total=len(expanded_arena_transforms)):
    if not (mouse_id, phase) in video_file_df.index:
        continue
    
    video_path = video_file_df.loc[(mouse_id, phase), "file_path"]
    total_frame_count = int(get_cv2_video_properties(video_path, cv2.CAP_PROP_FRAME_COUNT)[0])
    selection_frame_indices = [selection_frame_index, int(total_frame_count*0.9)]
    for selection_frame_index in selection_frame_indices:
        verification_frame = get_frames(video_path, [selection_frame_index], jump_to_frames=True)[0]

        arena_transform = np.reshape(flattened_arena_transform, (3, 3))
        transformed_arena = perspective_transform_arena(arena_dict, arena_transform)
        canvas = create_canvas(*verification_frame.shape[:2][::-1], dpi=100)
        plot_arena(transformed_arena, ax=canvas.gca())

        annotated_verification_frame = add_rgba_overlay_to_frame(verification_frame.copy(), fig2cv2(canvas))
        plt.close(canvas)

        Image.fromarray(annotated_verification_frame).save(local_arena_frames_dir / pathlib.Path(video_path).name.replace(".mp4", f"_frame{selection_frame_index}_verification.png"))

100%|██████████| 48/48 [00:40<00:00,  1.19it/s]


# Rat Presence

## Old Data

In [90]:
local_annotations_dir = annotations_dir / "rat_presence" / "cohort1"
expanded_arena_transforms = pd.read_csv(local_annotations_dir / "expanded_arena_transforms.csv", index_col=[0, 1, 2])
arena_dict = load_arena_from_json(local_annotations_dir / "arena_dict.json")
expanded_arena_transforms

0         1      2  \
mouse_id phase      selection_frame_index                              
1_1      WithoutRat 0                      1.278213 -0.035558  286.0   
         WithRat    0                      1.250645 -0.042362  291.0   
1_2      WithoutRat 0                      1.293721 -0.089809  298.0   
         WithRat    0                      1.315071 -0.030735  290.0   
1_3      WithoutRat 0                      1.307814 -0.004739  245.0   
         WithRat    0                      1.339324  0.015303  242.0   
1_4      WithoutRat 0                      1.298473 -0.018776  247.0   
         WithRat    0                      1.325747  0.014004  241.0   
2_1      WithoutRat 0                      1.271762 -0.029599  291.0   
         WithRat    0                      1.243296 -0.014301  278.0   
2_2      WithoutRat 0                      1.239508 -0.053004  291.0   
         WithRat    0                      1.198457  0.049167  273.0   
2_3      WithoutRat 0                      1.287859  0.001139  246.0   
         WithRat    0                      1.291159 -0.000414  243.0   
2_4      WithoutRat 0                      1.276265 -0.023506  241.0   
         WithRat    0                      1.313613  0.014189  245.0   
3_1      WithoutRat 0                      1.239293 -0.026989  291.0   
         WithRat    0                      1.216190 -0.013461  279.0   
3_2      WithoutRat 0                      1.291082 -0.021913  292.0   
         WithRat    0                      1.202139 -0.021189  275.0   
3_3      WithoutRat 0                      1.272840 -0.010779  239.0   
         WithRat    0                      1.299992  0.037706  240.0   
3_4      WithoutRat 0                      1.280692 -0.029268  241.0   
         WithRat    0                      1.315436  0.051984  239.0   

                                                  3         4      5  \
mouse_id phase      selection_frame_index                              
1_1      WithoutRat 0                     -0.081891  1.360756  303.0   
         WithRat    0                     -0.116180  1.333222  316.0   
1_2      WithoutRat 0                     -0.001658  1.415342  275.0   
         WithRat    0                     -0.079262  1.370904  314.0   
1_3      WithoutRat 0                     -0.083300  1.413316  315.0   
         WithRat    0                     -0.082718  1.431914  314.0   
1_4      WithoutRat 0                     -0.093969  1.392030  317.0   
         WithRat    0                     -0.089857  1.414419  317.0   
2_1      WithoutRat 0                     -0.082910  1.371034  302.0   
         WithRat    0                     -0.146905  1.351556  323.0   
2_2      WithoutRat 0                     -0.098393  1.329695  302.0   
         WithRat    0                     -0.205524  1.374382  332.0   
2_3      WithoutRat 0                     -0.103168  1.423920  316.0   
         WithRat    0                     -0.099311  1.382206  316.0   
2_4      WithoutRat 0                     -0.085375  1.398281  311.0   
         WithRat    0                     -0.129042  1.384672  318.0   
3_1      WithoutRat 0                     -0.096680  1.377434  305.0   
         WithRat    0                     -0.138964  1.371250  314.0   
3_2      WithoutRat 0                     -0.066213  1.402094  305.0   
         WithRat    0                     -0.119630  1.399674  305.0   
3_3      WithoutRat 0                     -0.093311  1.433760  313.0   
         WithRat    0                     -0.149128  1.409918  327.0   
3_4      WithoutRat 0                     -0.077676  1.397942  313.0   
         WithRat    0                     -0.156882  1.421134  327.0   

                                                  6             7    8  
mouse_id phase      selection_frame_index                               
1_1      WithoutRat 0                     -0.000143 -4.093312e-05  1.0  
         WithRat    0                     -0.000161 -8.783526e-05  1.0  
1_2      Wit

In [91]:
local_videos_dir = videos_dir / "rat_presence" / "cohort1"
video_file_df = build_file_df(str(local_videos_dir), r"(?P<date_string>\d+)_mouse(?P<mouse_id>[\d_]+)_(?P<phase>\w+)_remuxed.mp4")
video_file_df = video_file_df.set_index(["mouse_id", "phase"])
video_file_df

date_string  \
mouse_id phase                    
1_1      WithoutRat    20210531   
         WithRat       20210531   
1_2      WithoutRat    20210531   
         WithRat       20210531   
2_1      WithoutRat    20210531   
         WithRat       20210531   
2_2      WithoutRat    20210531   
         WithRat       20210531   
3_1      WithoutRat    20210531   
         WithRat       20210531   
3_2      WithoutRat    20210531   
         WithRat       20210531   
1_3      WithoutRat    20210601   
         WithRat       20210601   
1_4      WithoutRat    20210601   
         WithRat       20210601   
2_3      WithoutRat    20210601   
         WithRat       20210601   
2_4      WithoutRat    20210601   
         WithRat       20210601   
3_3      WithoutRat    20210601   
         WithRat       20210601   
3_4      WithoutRat    20210601   
         WithRat       20210601   

                                                             file_path  
mouse_id phase                                                          
1_1      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_2      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
2_1      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
2_2      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
3_1      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
3_2      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_3      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_4      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
2_3      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
2_4      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
3_3      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
3_4      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...

In [92]:
local_arena_frames_dir = arena_frames_dir / "rat_presence" / "cohort1"
local_arena_frames_dir.mkdir(parents=True, exist_ok=True)

for (mouse_id, phase, selection_frame_index), flattened_arena_transform in tqdm(expanded_arena_transforms.iterrows(), total=len(expanded_arena_transforms)):
    if not (mouse_id, phase) in video_file_df.index:
        continue
    
    video_path = video_file_df.loc[(mouse_id, phase), "file_path"]
    total_frame_count = int(get_cv2_video_properties(video_path, cv2.CAP_PROP_FRAME_COUNT)[0])
    selection_frame_indices = [selection_frame_index, int(total_frame_count*0.9)]
    for selection_frame_index in selection_frame_indices:
        verification_frame = get_frames(video_path, [selection_frame_index], jump_to_frames=True)[0]

        arena_transform = np.reshape(flattened_arena_transform, (3, 3))
        transformed_arena = perspective_transform_arena(arena_dict, arena_transform)
        canvas = create_canvas(*verification_frame.shape[:2][::-1], dpi=100)
        plot_arena(transformed_arena, ax=canvas.gca())

        annotated_verification_frame = add_rgba_overlay_to_frame(verification_frame.copy(), fig2cv2(canvas))
        plt.close(canvas)

        Image.fromarray(annotated_verification_frame).save(local_arena_frames_dir / pathlib.Path(video_path).name.replace(".mp4", f"_frame{selection_frame_index}_verification.png"))

100%|██████████| 24/24 [00:16<00:00,  1.43it/s]


In [ ]:
_index_df = expanded_arena_transforms.index.to_frame()
_index_df["mouse_id"] = _index_df["mouse_id"].astype(str).str.replace(".", "_")
# _index_df.iloc[2:4].loc[:, "mouse_id"] = "1_10"
# _index_df.iloc[26:28].loc[:, "mouse_id"] = "3_10"
expanded_arena_transforms.index = pd.MultiIndex.from_frame(_index_df)
expanded_arena_transforms.to_csv(local_annotations_dir / "expanded_arena_transforms.csv")

## New Data

In [105]:
local_annotations_dir = annotations_dir / "rat_presence" / "cohort2"
expanded_arena_transforms = pd.read_csv(local_annotations_dir / "expanded_arena_transforms.csv", index_col=[0, 1, 2])
arena_dict = load_arena_from_json(local_annotations_dir / "arena_dict.json")
expanded_arena_transforms

0         1      2  \
mouse_id phase      selection_frame_index                              
1_1      WithoutRat 0                      2.321970 -0.044540  645.0   
         WithRat    0                      2.371868 -0.007890  644.0   
1_10     WithoutRat 0                      2.345324 -0.029796  596.0   
         WithRat    0                      2.266315 -0.049217  599.0   
1_11     WithoutRat 0                      2.266598 -0.034004  597.0   
         WithRat    0                      2.329333 -0.030440  596.0   
1_12     WithoutRat 0                      2.266006 -0.049819  599.0   
         WithRat    0                      2.389973  0.013054  593.0   
1_2      WithoutRat 0                      2.274688 -0.053577  646.0   
         WithRat    0                      2.294768 -0.009686  651.0   
1_3      WithoutRat 0                      2.302884 -0.008773  649.0   
         WithRat    0                      2.326501 -0.000036  650.0   
1_4      WithoutRat 0                      2.282354  0.012691  649.0   
         WithRat    0                      2.323293 -0.005267  650.0   
1_5      WithoutRat 0                      2.353171 -0.011659  591.0   
         WithRat    0                      2.345946  0.018764  591.0   
1_6      WithoutRat 0                      2.396086 -0.056196  593.0   
         WithRat    0                      2.350337 -0.021066  595.0   
1_7      WithoutRat 0                      2.345137 -0.012235  589.0   
         WithRat    0                      2.355769 -0.001959  588.0   
1_8      WithoutRat 0                      2.328685  0.001752  591.0   
         WithRat    0                      2.377481  0.036902  589.0   
1_9      WithoutRat 0                      2.344150 -0.030242  593.0   
         WithRat    0                      2.346703  0.017560  594.0   
3_1      WithoutRat 0                      2.240318 -0.018406  603.0   
         WithRat    0                      2.248648  0.012891  611.0   
3_10     WithoutRat 0                      2.189332  0.011791  664.0   
         WithRat    0                      2.317412  0.001174  667.0   
3_11     WithoutRat 0                      2.273336  0.050364  665.0   
         WithRat    0                      2.247961  0.011326  671.0   
3_12     WithoutRat 0                      2.256609  0.031757  665.0   
         WithRat    0                      2.374109  0.045795  666.0   
3_2      WithoutRat 0                      2.253740  0.025657  609.0   
         WithRat    0                      2.215394 -0.030587  614.0   
3_3      WithoutRat 0                      2.268721  0.011383  611.0   
         WithRat    0                      2.231697 -0.026299  614.0   
3_4      WithoutRat 0                      2.259150  0.041436  629.0   
         WithRat    0                      2.300757  0.066324  624.0   
3_5      WithoutRat 0                      2.288794 -0.004690  669.0   
         WithRat    0                      2.230712 -0.008226  673.0   
3_6      WithoutRat 0                      2.297336 -0.031525  673.0   
         WithRat    0                      2.269257  0.041024  671.0   
3_7      WithoutRat 0                      2.260483  0.022671  659.0   
         WithRat    0                      2.226645  0.013648  661.0   
3_8      WithoutRat 0                      2.280661 -0.023314  665.0   
         WithRat    0                      2.197720  0.027298  663.0   
3_9      WithoutRat 0                      2.281015  0.043232  663.0   
         WithRat    0                      2.285096  0.027748  664.0   

                                                  3         4      5  \
mouse_id phase      selection_frame_index                              
1_1      WithoutRat 0                      0.040138  2.199943  218.0   
         WithRat    0                      0.057129  2.248069  215.0   
1_10     WithoutRat 0                      0.061278  2.218926  222.0   
         WithRat    0                      0.048583  2.187367  222.0   
1_11     WithoutR

In [106]:
local_videos_dir = videos_dir / "rat_presence" / "cohort2"
video_file_df = build_file_df(str(local_videos_dir), r"(?P<date_string>\d+)_mouse(?P<mouse_id>[\d_]+)_(?P<phase>\w+)_remuxed.mp4")
video_file_df = video_file_df.set_index(["mouse_id", "phase"])
video_file_df

date_string  \
mouse_id phase                    
1_1      WithoutRat    20241115   
         WithRat       20241115   
1_2      WithoutRat    20241115   
         WithRat       20241115   
1_3      WithoutRat    20241115   
         WithRat       20241115   
1_4      WithoutRat    20241115   
         WithRat       20241115   
1_5      WithoutRat    20241115   
         WithRat       20241115   
1_6      WithoutRat    20241115   
         WithRat       20241115   
1_10     WithoutRat    20241116   
         WithRat       20241116   
1_11     WithoutRat    20241116   
         WithRat       20241116   
1_12     WithoutRat    20241116   
         WithRat       20241116   
1_7      WithoutRat    20241116   
         WithRat       20241116   
1_8      WithoutRat    20241116   
         WithRat       20241116   
1_9      WithoutRat    20241116   
         WithRat       20241116   
3_1      WithoutRat    20241122   
         WithRat       20241122   
3_2      WithoutRat    20241122   
         WithRat       20241122   
3_3      WithoutRat    20241122   
         WithRat       20241122   
3_4      WithoutRat    20241122   
         WithRat       20241122   
3_5      WithoutRat    20241122   
         WithRat       20241122   
3_6      WithoutRat    20241122   
         WithRat       20241122   
3_10     WithoutRat    20241123   
         WithRat       20241123   
3_11     WithoutRat    20241123   
         WithRat       20241123   
3_12     WithoutRat    20241123   
         WithRat       20241123   
3_7      WithoutRat    20241123   
         WithRat       20241123   
3_8      WithoutRat    20241123   
         WithRat       20241123   
3_9      WithoutRat    20241123   
         WithRat       20241123   

                                                             file_path  
mouse_id phase                                                          
1_1      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_2      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_3      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_4      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_5      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_6      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_10     WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_11     WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_12     WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_7      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_8      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
1_9      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
3_1      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
3_2      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
         WithRat     D:\Folder With Backup\2019_paper\data\videos_r...  
3_3      WithoutRat  D:\Folder With Backup\2019_paper\data\videos_r...  
      

In [ ]:
local_arena_frames_dir = arena_frames_dir / "rat_presence" / "cohort2"
local_arena_frames_dir.mkdir(parents=True, exist_ok=True)

for (mouse_id, phase, selection_frame_index), flattened_arena_transform in tqdm(expanded_arena_transforms.iterrows(), total=len(expanded_arena_transforms)):
    if not (mouse_id, phase) in video_file_df.index:
        continue
    
    video_path = video_file_df.loc[(mouse_id, phase), "file_path"]
    total_frame_count = int(get_cv2_video_properties(video_path, cv2.CAP_PROP_FRAME_COUNT)[0])
    selection_frame_indices = [selection_frame_index, int(total_frame_count*0.9)]
    for selection_frame_index in selection_frame_indices:
        verification_frame = get_frames(video_path, [selection_frame_index], jump_to_frames=True)[0]

        arena_transform = np.reshape(flattened_arena_transform, (3, 3))
        transformed_arena = perspective_transform_arena(arena_dict, arena_transform)
        canvas = create_canvas(*verification_frame.shape[:2][::-1], dpi=100)
        plot_arena(transformed_arena, ax=canvas.gca())

        annotated_verification_frame = add_rgba_overlay_to_frame(verification_frame.copy(), fig2cv2(canvas))
        plt.close(canvas)

        Image.fromarray(annotated_verification_frame).save(local_arena_frames_dir / pathlib.Path(video_path).name.replace(".mp4", f"_frame{selection_frame_index}_verification.png"))

100%|██████████| 48/48 [00:40<00:00,  1.18it/s]


: 